# Introduction 

This notebook is devoted to the the physical and mathematical model for the operations of the thermal generators. The outline of this notebook is of follows : 

1. Physical Description of the generators 
2. Mathematical Model for the generators 
3. Syntax for `Thermal_Generator`

# 1. Physical Description for the generators 

Thermal Generators, or *Thermal Power Station, is a power station in which heat energy is converted into electric power* [Wiki](https://en.wikipedia.org/wiki/Thermal_power_station#Types_of_thermal_energy). Thermal generators operate on different type of fuels (coal, natural gas, petroleum, nuclear, ...), but for the initial purpose of this model, coal and gas are the main fuel sources of concern. 

## 1.1 Market Description 

The `Thermal_Generator` assumes that the generators participate in the electricity day-ahead market, in which the generator operator would have to optimize their operation strategy based on their given as well as forecasted information (electricity, fuel, carbon prices) in order to determine the optimal dispatch strategy for the generator (when to start up, operate in low/high power mode, when to shut down). 

Data regarding the markets must be provided to the model, including: electricity, fuel and carbon price for the specified time interval (the time resolution must be in the unit of one hour). 

- Electricity price ($€/MWh_{e}$): this is the price on the electricity day-ahead market. For the date of the markets participating on the *EPEX SPOT* power exchange, the `EPEX_Scrapping` module is design to help you automatically acquire the data given on their website.

- Fossil Fuel Price ($€/MWh_{th}$): this is the price of fossil fuels (some time the energy unit is in Btu, but for the purpose of simplicity, we use MWh thermal as the standard unit)

- Carbon Price ($€/tCO_{2}$ or $€/EA$, $EA = 1 \text{ tonne of } CO_{2}$ ) : carbon emission price, they will be traded in the unit of *Emission Allowance (EA)*, which is equivalent to one tonne of $CO_{2}$. 

Apart from the EPEX SPOT, other useful sources of data can be found in : [EEX](https://www.eex.com/en#/en), [Quandl](https://www.quandl.com/) or [ICE](https://www.theice.com/futures-europe). 


## 1.2 Technical Operation and Expenses Description 

Depending on the comparison between the price of electricity and the short-term marginal cost for the time horizon of interest, the generator's operator will decide to turn on the generator (Cold Start-up), shut down or switching between different generation modes (low, high, maximum powers with different efficiencies). The objective is to minimize the operational cost as well as maximize the revenue coming from the electricity day-ahead market (DAM). 

- Start-up : cold startup is the a very costly operation with a thermal generator (~ 40,000 € for a coal-fired generator, depending on the nominal power). If the operator decides to turn on the generator, the generator cannot generate electricity within a certain amount of time (`Startup_cold_time`), typical value for a Combined Cycle Gas Turbine (CCGT) is 2 hours and 10 for coal. The formula for start-up cost is as follow ([ref](https://www.stiftung-mercator.de/media/downloads/3_Publikationen/nature_energy_Start-up_costs__of_thermal_power_plants.pdf), [ref](https://www.diw.de/documents/publikationen/73/diw_01.c.524200.de/dp1540.pdf))

\begin{align}
    \text{Startup Cost }(€/MW_{nominal}) = \text{required Startup fuel } (MWh_{th}/MW_{nominal}) \times \text{fuel price } (€/MWh_{th}) + \text{Startup Depreciation Cost } (€/MW_{nominal})
\end{align}

- Shutdown : in this model, shutdown does not generate expenses but does put a minimum off time constraint on the generator, which means that the generator can't be operational for at least `Minimum_downtime` hours from the moment where the shutdown signal is made.

- Operation modes : the generators can operate on different modes with different power with different efficiencies (e.g. operates at 40% nominal power to minimize losses when DAM price is low, but not that low to be shut down completely). Efficiency will be lower than the nominal one (thermodynamics!). 

- Capacity Factor : determines the maximum number of hours that the generator can be operational (given in the unit of the fraction of a year)


Short-term Marginal Cost (STMC) : the cost of short-term operation 


\begin{align}
    \text{STMC }(€/MW_{nominal}) = \left[ \text{Fuel Price } (€/MWh_{e}) +  CO_{2}\text{ Price} (€/MWh_{e}) + \text{Variable O&M} (€/MWh_{e}) \right]\times \text{ Energy } (€/MWh_{e}) 
\end{align}


$\qquad \qquad$ where: 


\begin{align}
    \text{Fuel Price } (€/MWh_{e}) &= \frac{\text{Fossil Fuel Price } (€/MWh_{th})} {\text{ Efficiency } (MWh_{e}/MWh_{th})} \\ \\
    CO_{2} \text{ Price } (€/MWh_{e}) &= \frac{\text{ Emission Factor } (tCO_{2}MWh_{th})}{\text{ Efficiency } (MWh_{e}/MWh_{th})} \times { Carbon Price } (€/tCO_{2})
\end{align}

# 2. Mathematical Model 

In this section, we will describe the mathematical formulation for the optimization model used in `Thermal_Generator`, which uses [`optlang`](https://pypi.org/project/optlang/), an optimization modelling language that generates symbolic formulation for the problem.  

## 2.1 Formulation : 

The formulation is as follows: 

\begin{align}
    \max_{\mathbf{X},\mathbf{S}, \mathbf{F}} \qquad \sum_{t=1}^{H} \left[ ( DAM_{t} - STMC_{t}) \sum_{m=1}^{M} E^{m} X_{t}^{m} \right] -  P_{nominal} \sum_{t=1}^{H} \left[ \text{Startup_dep_cost} + \text{Startup_fuel}\times\text{price_fossil}_{t} \right]S_{t}
\end{align}

$\qquad$ s.t.

\begin{align}
      X_{t}^{m} \in \{0, 1\} \qquad &\forall t \in [1, 2, ..., H],  m \in [1, 2, ..., M] \\
      S_{t} \in \{0, 1\} \qquad &\forall t \in [1, 2, ..., H] \\
      F_{t} \in \{0, 1\} \qquad &\forall t \in [1, 2, ..., H] \\
      X_{t}^{m} = 0 \qquad &\forall t \in [1, 2, ..., \alpha],  m \in [1, 2, ..., M] \\
    \sum_{m=1}^{M} X_{t}^{m} \leq 1 \qquad &\forall t \in [1, 2, ..., H] \\
    S_{t} + F_{t} \leq 1 \qquad &\forall t \in [1, 2, ..., H] \\
    S_{t-\alpha} - F_{t} = \sum_{m=1}^{M} (X_{t+1}^{m} - X_{t}^{m}) \qquad  &\forall t \in [\alpha, \alpha + 1, ..., H - 1], \qquad \alpha \text{ is the startup time} \\
    \alpha (1 - S_{t}) \geq \sum_{m=1}^{M} \sum_{k=1}^{\alpha} X_{t+k}^{m} \qquad &\forall t \in [1, 2, ..., H-\alpha] \\
    \beta (1 - F_{t}) \geq \sum_{m=1}^{M} \sum_{k=1}^{\beta} X_{t+k}^{m} \qquad &\forall t \in [1, 2, ..., H-\beta], \qquad \beta \text{ is the minimum offline after shutdown} \\ 
    \sum_{t=1}^{H} \sum_{m=1}^{m} X_{t}^{m} \leq 24\times 365 \times CF    & \qquad ,CF \text{ is the Capacity Factor}
\end{align}

$\qquad \qquad$ where,

- $H$ is the time horizon (in hours), $M$ is the number of operation modes
- $X_{t}^{m} \in \{0, 1\}$ is the binary variable represents the state of the operator at time $t$ and mode $m$. $1$ means that the generator is in mode $m$ at time $t$, 0 otherwise.
- $S_{t}$ represents the state of the startup process of the generator, $1$ means the startup signal is at time $t$, 0 otherwise. 
- $F_{t}$ represents the state of the shutdown process of the generator, $1$ means the shutdown signal is at time $t$, 0 otherwise. 
- $E^{m} = 1 \times P^{m}$ is the electric energy generated within one hour in mode `m`. 

## 2.2 Interpretation of the formulation 

The physical interpretation of the constraints above are described here. 

$X_{t}^{m} \in \{0, 1\}$ : binary variable statement 

$X_{t}^{m} = 0, \qquad \forall t \in [1, 2, ..., \alpha]$ : offline initial states condition, which is to ensure that it has to wait for a start-up signal before coming online from the beginning of the horizon. 

$S_{t} + F_{t} \leq 1, \qquad \forall t \in [1, 2, ..., H]$ : makes sure that start-up and shutdown signals can't be at the same time

$S_{t-\alpha} - F_{t} = \sum_{m=1}^{M} (X_{t+1}^{m} - X_{t}^{m}), \qquad  \forall t \in [\alpha, \alpha + 1, ..., H - 1]$ : start-up and shutdown logical constraints which execute the startup or shutdown signals at the corresponding time points. 

$\alpha (1 - S_{t}) \geq \sum_{m=1}^{M} \sum_{k=1}^{\alpha} X_{t+k}^{m}, \qquad \forall t \in [1, 2, ..., H-\alpha]$ : cold startup time requirement

$\beta (1 - F_{t}) \geq \sum_{m=1}^{M} \sum_{k=1}^{\beta} X_{t+k}^{m}, \qquad \forall t \in [1, 2, ..., H-\beta]$ : minimum downtime requirement 

$\sum_{t=1}^{H} \sum_{m=1}^{m} X_{t}^{m} \leq 24\times 365 \times CF$ : capacity factor constraint 

## 2.3 Operation and Economic Indicators 

The metrics of the generator are stored in `.Operation_Profile` and `.Finance_Metrics` attributes in forms of dictionary. These metrics specify the dispatch profile as of the generator well as its balance sheet.

#### Economics Indicators (`.Finance_Metrics`)

$ \text{Revenue} = \sum_{t=1}^{H} ( DAM_{t} \times \sum_{m=1}^{M} X_{t}^{m} \times 1$ ) : revenue coming from the eletricity DAM.

$\text{OPEX} = \sum{t}^{H} (\sum_{m}^{M} STMC_{t}^{m} + \text{Start-up}_{t})+ \text{Fixed O&M}$ : operating expense, including the STMC, startup cost and fixed O&M cost ($€/MW_{nominal}/year$) 

$\text{Gross Profit} = \text{Revenue} - \text{OPEX} $ income before tax

$\text{Average STMC}$ : average STMC over the time horizon

#### Operation_Profile (`.Operation_Profile`)

$\text{Energy_Profile}$ : energy generation profile through out the time horizon

$\text{Capacity Factor} = \frac{\sum_{t=1}^{H} \sum_{m=1}^{M} X_{t}^{m}}{24\times 365}$ : the actual time of operation per year 

$\text{Number of Startups} = \sum_{t=1}^{H} S_{t}$ 


# 3. Syntax 

### Input : 

The input for the function `Thermal_Generator` can be passed in the form of a dictionary, as seen below. Notice that : `price` and `Name` are compulsory inputs. The other inputs are assigned with the defaults values as seen in the `config` dictionaries  

In [4]:
import pandas as pd
gas_uk = pd.read_csv('data/CCGT_UK.csv', index_col='time', parse_dates=True)
coal_uk = pd.read_csv('data/Coal_UK.csv', index_col='time', parse_dates=True)

In [5]:
config_gas = {'price': gas_uk, 'Name': 'CCGT_UK', 'Efficiency': [0.58, 0.47], 'Power': [100, 40], 
             'Emission_Factor': 0.25, 'CF': 0.75, 'Startup_cold_time': 2, 'Startup_dep_cost': 60,
             'Startup_fuel': 2.8, 'Minimum_downtime': None,
             'Cost_fixed_OM': 28100, 'Cost_var_OM': 2.3}

config_coal = {'price': coal_uk, 'Name': 'Coal_UK', 'Efficiency': [0.44, 0.38], 'Power': [100, 40], 
             'Emission_Factor': 0.34, 'CF': 0.6, 'Startup_cold_time': 10, 'Startup_dep_cost': 105,
             'Startup_fuel': 2.7, 'Minimum_downtime': None, 'Cost_fixed_OM': 38000, 'Cost_var_OM': 4.04}

**`price`** : a DataFrame with DatetimeIndex that contains the commodities' prices, which have already been described in **Market Description**

In [3]:
gas_uk.head()

,fossil,electricity,carbon
time,,,
2017-01-01 00:00:00,16.4886,60.21,20.506706
2017-01-01 01:00:00,16.4886,60.71,20.506706
2017-01-01 02:00:00,16.4886,75.97,20.506706
2017-01-01 03:00:00,16.4886,60.74,20.506706
2017-01-01 04:00:00,16.4886,49.00,20.506706


**`Name`** : is the name assigned to the optimization model (`.optim_model`) and it must be unique

`Efficiency`, `Power` : lists containing powers and their corresponding efficiencies for different modes of operation. The first elements of these lists must be the values of the nominal power. `Power` has the unit of MW, which is the standard unit for power used in `Thermal_Generator`. 

`Emission_Factor` ($tCO_{2}/MWh_{th}$) : the amount of Green House Gas emitted per unit of fuel consumed for the each technology

`CF` : Nominal Capacity Factor

`Startup_cold_time` : number of hours required for cold start-up

`Startup_dep_cost` ($€/MW_{nominal}$): depreciation cost for each cold start-up 

`Startup_fuel` ($MWh_{th}/MW_{nominal}$) : fuel consumed for each cold start-up

`Minimum_downtime` : minimum offtime after the shutdown signals (hours). This is the parameter $\beta$ described above, which correpsonds to the minimum downtime constraint. The minimum downtime constraint is not genuinely necessary because the solver will not decide to turn off the generator unless the market conditions (prices) are not favourable for the upcoming hours. Activate this constraint can cause the troubles in terms of time for the problem initialization and solving. 

`Cost_fixed_OM` $(€/MW_{nominal}/year)$: the fixed O&M cost. If the time interval is less than a year, a fraction of this cost ($\times \frac{Horizon}{8760}$)

`Cost_var_OM` $(€/MWh_{e})$: variable O&M cost

### Tutorials and Output

The instruction for tutorials and output is discused in the `Tutorials` notebook. 